# Eliminação Recursiva de Atributos (RFE) - Implantação

Esse componente executa a classificação de atributos com eliminação recursiva de atributos com base em um estimador do Random Forest com hiperparâmetros padrão. A validação cruzada, K-fold, é empregada para estimar a importância do recurso. Faz uso da implementação `RFECV` do [Scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFECV.html). <br>
Scikit-learn é uma biblioteca open source de machine learning que suporta apredizado supervisionado e não supervisionado. Também provê várias ferramentas para montagem de modelo, pré-processamento de dados, seleção e avaliação de modelos, e muitos outros utilitários.

Este notebook apresenta:
- como carregar modelos e outros resultados do treinamento.
- como declarar parâmetros e usá-los para criar componentes reutilizáveis.

## Declaração de Classe para Predições em Tempo Real

A tarefa de implantação cria um serviço REST para predições em tempo real.<br>
Para isso você deve criar uma classe `Model` que implementa o método `predict`.

In [ ]:
%%writefile Model.py
import joblib
import numpy as np


class Model(object):
    def __init__(self):
        # Carrega artefatos: estimador, etc
        artifacts = joblib.load("/tmp/data/rfe-selector.joblib")
        self.pipeline = artifacts["pipeline"]
        self.selected_features = artifacts["selected_features"]

    def class_names(self):
        return self.selected_features

    def predict(self, X, feature_names, meta=None):
        return self.pipeline.transform(X)

## Teste do serviço REST

Crie um arquivo `contract.json` com os seguintes atributos:

- `features` : A lista de features em uma requisição.
- `targets` : A lista de valores retornados pelo método `predict`.

Cada `feature` pode conter as seguintes informações:

- `name` : nome da feature
- `ftype` : tipo da feature : **continuous** ou **categorical**
- `dtype` : tipo de dado : **FLOAT** ou **INT** : *obrigatório para ftype continuous*
- `range` : intervalo de valores numéricos : *obrigatório para ftype continuous*
- `values` : lista de valores categóricos : *obrigatório para ftype categorical*

Em seguida, utilize a função `test_deployment` do [SDK da PlatIAgro](https://platiagro.github.io/sdk/) para simular predição em tempo-real.<br>

In [ ]:
%%writefile contract.json
{
    "features": [
        {
            "name": "crim",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [0.0, 100.0]
        },
        {
            "name": "zn",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [0.0, 100.0]
        },
        {
            "name": "indus",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [0.0, 100.0]
        },
        {
            "name": "chas",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [0.0, 1.0]
        },
        {
            "name": "nox",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [0.0, 1.0]
        },
        {
            "name": "rm",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [0.0, 10.0]
        },
        {
            "name": "age",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [0.0, 100.0]
        },
        {
            "name": "dis",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [0.0, 100.0]
        },
        {
            "name": "rad",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [1.0, 24.0]
        },
        {
            "name": "tax",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [0.0, 1000.0]
        },
        {
            "name": "ptratio",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [0.0, 100.0]
        },
        {
            "name": "black",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [0.0, 1000.0]
        },
        {
            "name": "lstat",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [0.0, 100.0]
        }
    ],
    "targets": [
        {
            "name": "estimate",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [0.0, 100.0]
        }
    ]
}

In [ ]:
from platiagro.deployment import test_deployment

test_deployment("contract.json")